In [8]:
pip install loguru

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
from typing import Any, Dict, Optional

from loguru import logger


@logger.catch(reraise=True)
def coco_merge(
    input_extend: str,
    input_add: str,
    output_file: str,
    indent: Optional[int] = None,
) -> str:
    """Merge COCO annotation files.

    Args:
        input_extend: Path to input file to be extended.
        input_add: Path to input file to be added.
        output_file : Path to output file with merged annotations.
        indent: Argument passed to `json.dump`. See https://docs.python.org/3/library/json.html#json.dump.
    """
    with open(input_extend, "r") as f:
        data_extend = json.load(f)
    with open(input_add, "r") as f:
        data_add = json.load(f)

    output: Dict[str, Any] = {
        k: data_extend[k] for k in data_extend if k not in ("images", "annotations")
    }

    output["images"], output["annotations"] = [], []

    for i, data in enumerate([data_extend, data_add]):
        logger.info(
            "Input {}: {} images, {} annotations".format(
                i + 1, len(data["images"]), len(data["annotations"])
            )
        )

        cat_id_map = {}
        for new_cat in data["categories"]:
            new_id = None
            for output_cat in output["categories"]:
                if new_cat["name"] == output_cat["name"]:
                    new_id = output_cat["id"]
                    break

            if new_id is not None:
                cat_id_map[new_cat["id"]] = new_id
            else:
                new_cat_id = max(c["id"] for c in output["categories"]) + 1
                cat_id_map[new_cat["id"]] = new_cat_id
                new_cat["id"] = new_cat_id
                output["categories"].append(new_cat)

        img_id_map = {}
        for image in data["images"]:
            n_imgs = len(output["images"])
            img_id_map[image["id"]] = n_imgs
            image["id"] = n_imgs

            output["images"].append(image)

        for annotation in data["annotations"]:
            n_anns = len(output["annotations"])
            annotation["id"] = n_anns
            annotation["image_id"] = img_id_map[annotation["image_id"]]
            annotation["category_id"] = cat_id_map[annotation["category_id"]]

            output["annotations"].append(annotation)

    logger.info(
        "Result: {} images, {} annotations".format(
            len(output["images"]), len(output["annotations"])
        )
    )

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=indent, ensure_ascii=False)

    return output_file

In [4]:
import os

In [23]:
coco_paths = sorted([path for path in os.listdir('../data/Уставные грамоты/Обработка/размеченные') if 'coco' in path.split('.')[-2]])

In [24]:
len(coco_paths)

78

In [25]:
base_path = '/home/admin01/vadim/historic_docs_19/data/Уставные грамоты/Обработка/размеченные/'
input_extend =  base_path + coco_paths[0]
input_add = base_path + coco_paths[1]
output_file = 'train_annotation.json'

coco_merge(input_extend, input_add, output_file, indent=None)

for coco_path in coco_paths[2:-14]:
    input_add = base_path + coco_path
    coco_merge(output_file, input_add, output_file, indent=None)
    

2023-12-05 05:08:10.628 | INFO     | __main__:coco_merge:36 - Input 1: 2 images, 60 annotations
2023-12-05 05:08:10.630 | INFO     | __main__:coco_merge:36 - Input 2: 4 images, 79 annotations
2023-12-05 05:08:10.631 | INFO     | __main__:coco_merge:74 - Result: 6 images, 139 annotations
2023-12-05 05:08:10.646 | INFO     | __main__:coco_merge:36 - Input 1: 6 images, 139 annotations
2023-12-05 05:08:10.647 | INFO     | __main__:coco_merge:36 - Input 2: 4 images, 78 annotations
2023-12-05 05:08:10.649 | INFO     | __main__:coco_merge:74 - Result: 10 images, 217 annotations
2023-12-05 05:08:10.668 | INFO     | __main__:coco_merge:36 - Input 1: 10 images, 217 annotations
2023-12-05 05:08:10.668 | INFO     | __main__:coco_merge:36 - Input 2: 1 images, 39 annotations
2023-12-05 05:08:10.669 | INFO     | __main__:coco_merge:74 - Result: 11 images, 256 annotations
2023-12-05 05:08:10.678 | INFO     | __main__:coco_merge:36 - Input 1: 11 images, 256 annotations
2023-12-05 05:08:10.679 | INFO   

Итого для обучения возможно использовать 502 изображений и 13282 (возможно строка)

In [26]:
base_path = '/home/admin01/vadim/historic_docs_19/data/Уставные грамоты/Обработка/размеченные/'
input_extend =  base_path + coco_paths[-14]
input_add = base_path + coco_paths[-13]
output_file = 'val_annotation.json'

coco_merge(input_extend, input_add, output_file, indent=None)

for coco_path in coco_paths[-12:]:
    input_add = base_path + coco_path
    coco_merge(output_file, input_add, output_file, indent=None)

2023-12-05 05:08:19.945 | INFO     | __main__:coco_merge:36 - Input 1: 5 images, 96 annotations
2023-12-05 05:08:19.947 | INFO     | __main__:coco_merge:36 - Input 2: 6 images, 115 annotations
2023-12-05 05:08:19.949 | INFO     | __main__:coco_merge:74 - Result: 11 images, 211 annotations
2023-12-05 05:08:19.964 | INFO     | __main__:coco_merge:36 - Input 1: 11 images, 211 annotations
2023-12-05 05:08:19.964 | INFO     | __main__:coco_merge:36 - Input 2: 7 images, 137 annotations
2023-12-05 05:08:19.965 | INFO     | __main__:coco_merge:74 - Result: 18 images, 348 annotations
2023-12-05 05:08:19.977 | INFO     | __main__:coco_merge:36 - Input 1: 18 images, 348 annotations
2023-12-05 05:08:19.978 | INFO     | __main__:coco_merge:36 - Input 2: 1 images, 20 annotations
2023-12-05 05:08:19.978 | INFO     | __main__:coco_merge:74 - Result: 19 images, 368 annotations
2023-12-05 05:08:19.986 | INFO     | __main__:coco_merge:36 - Input 1: 19 images, 368 annotations
2023-12-05 05:08:19.986 | INF

Итого для валидации возможно использовать 58 изображений и 1327 (возможно строка)